In [45]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pytz
from datetime import datetime
from binance_api import api_key, api_secret
import time
import ccxt
import pandas as pd
from datetime import timedelta

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()

def get_current_dataframe_and_last_5_volume_sum():
    symbol = 'ETH/USDT'

    # Create an instance of the Binance exchange
    exchange = ccxt.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    # Replace 'YOUR_START_TIMESTAMP' and 'YOUR_END_TIMESTAMP' with the desired timestamps
    current_time_utc = datetime.utcnow() - timedelta(hours=1)

    # Format the current time as a string in the required format ('%Y-%m-%dT%H:%M:%SZ')
    start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Parse the formatted string to create the start_timestamp
    start_timestamp = exchange.parse8601(start_time_str)
    # Fetch historical OHLCV data for the specified trading pair and timeframe
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)
    df['last_5_volume_sum'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']
    last_5_volume_sum = df['last_5_volume_sum'].iloc[-1]
    # Print the DataFrame
    return df, last_5_volume_sum

async def fetch_and_print_ohlcv(exchange, symbol, df_func):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')

    try:
        previous_timestamp = None  # Variable to store the previous timestamp

        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')

            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')
            current_volume = ohlcv[-1][5]

            # Check if the timestamp has changed
            if current_timestamp_str != previous_timestamp:
                print(f"New timestamp: {current_timestamp_str}")
                df, last_5_volume_sum = df_func()

                previous_timestamp = current_timestamp_str  # Update the previous timestamp

            if current_volume > last_5_volume_sum:
                
                print(f"last_5_volume_sum: {last_5_volume_sum}, current_volume: {current_volume}")

            print(f"time: {current_timestamp_str}, open: {ohlcv[-1][1]}, high: {ohlcv[-1][2]}, low: {ohlcv[-1][3]}, close: {ohlcv[-1][4]}, volume: {ohlcv[-1][5]}, last_5_volume_sum: {last_5_volume_sum:.2f}")

            await asyncio.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def main():
    symbol = 'ETH/USDT'
    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True 
        }
    })

    await fetch_and_print_ohlcv(exchange, symbol, get_current_dataframe_and_last_5_volume_sum)

# Run the async/await code in a Jupyter Notebook cell
asyncio.run(main())


New timestamp: 2023-09-29 01:04:00
time: 2023-09-29 01:04:00, open: 1658.4, high: 1658.7, low: 1657.7, close: 1657.7, volume: 1578.672, last_5_volume_sum: 11174.94


KeyboardInterrupt: 

In [105]:
import ccxt

api_key = '7f31555ee801f493d8df0beb6916b5c83642738ddcbc8ec92c1a17e82d0c949a'
api_secret = '7c497666094642f29ac66cc67abe25ea0c80dfe1bd79f6597dcff0358d7c256d'
symbol = 'ETH/USDT'

exchange = ccxt.binance({
    'options': {
        'defaultType': 'future',
        'newUpdates': True 
    },
    'apiKey': api_key,
    'secret': api_secret
})
exchange.set_sandbox_mode(True)

class SendOrders:
    def __init__(self, exchange, symbol, leverage=100):
        self.exchange = exchange
        self.symbol = symbol
        self.leverage = leverage
        self.test_mode = True

    def check_balance(self):
        if self.test_mode:
            return 100
        balance = self.exchange.fetch_balance()
        return balance['total']['USDT']

    def retrieve_price(self):
        ticker = self.exchange.fetch_ticker(self.symbol)
        return ticker['last']

    def retrieve_order_book(self):
        order_book = self.exchange.fetch_order_book(self.symbol)
        return order_book

    def order_book_bid_and_ask(self):
        order_book = self.retrieve_order_book()
        return order_book['bids'][0][0], order_book['asks'][0][0]

    def calculate_quantity(self):
        balance = self.check_balance()
        leveraged_balance = min(balance * self.leverage, 100000)
        quantity = leveraged_balance / self.retrieve_price()
        return quantity

    def create_1st_post_only_order(self, side):
        # side could be 'buy' or 'sell'
        quantity = self.calculate_quantity()
        bid_1st, ask_1st = self.order_book_bid_and_ask()
        price = bid_1st if side == 'buy' else ask_1st

        try:
            order = self.exchange.create_limit_order(
                self.symbol,
                side,
                quantity,
                price,
                {
                    'postOnly': True,
                    'timeInForce': 'PO'  # Use 'GTC' for Good 'Til Canceled
                }
            )
        except ccxt.BaseError as e:
            print(f"An error occurred: {e}")

        return order

    def create_1st_market_order(self, side):
        # side could be 'buy' or 'sell'
        quantity = self.calculate_quantity()
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': False
            }
        )
        return order

    def detect_position_exist(self):
        position = self.exchange.fetch_positions([self.symbol])
        return len(position) > 0
    
    def create_2nd_tp_and_sl_order(self):
        try:
            # Check if a position exists
            if self.detect_position_exist():
                # Get position information
                position = self.exchange.fetch_positions([self.symbol])
                if not position:
                    raise Exception("No position information found.")

                quantity = position[0]['info']['positionAmt']
                # make quantity positive
                quantity = abs(float(quantity))
                print(f"quantity: {quantity}")

                

                # Determine the order side (buy or sell)
                side = 'sell' if position[0]['side'] == 'long' else 'buy'

                # Calculate take profit and stop loss prices
                entry_price = float(position[0]['info']['entryPrice'])
                take_profit_price = entry_price * 1.0007 if side == 'sell' else entry_price * 0.9993
                
                stop_loss_price = entry_price * 0.9993 if side == 'sell' else entry_price * 1.0007

                # Create a post-only take profit order
                try:
                    create_take_profit_order = self.exchange.create_limit_order(
                        symbol=self.symbol,
                        side=side,
                        price=take_profit_price,
                        amount=quantity,  # Specify quantity here
                        params={
                            'postOnly': True,
                            'timeInForce': 'PO',
                            'reduceOnly': True  # Use 'GTC' for Good 'Til Canceled
                        })
                except ccxt.BaseError as e:
                    print(f"An error occurred in create_take_profit_order : {e}")

                try:
                    stop_loss_market_order = self.exchange.create_order(
                    symbol=self.symbol,
                    type='stop_market',
                    side=side,
                    amount=quantity,  # Specify quantity here
                    params={
                        'stopPrice': stop_loss_price,
                        'reduceOnly': True
                    })
                # Create a post-only take profit order
                except ccxt.BaseError as e:
                    print(f"An error occurred in stop_loss_market_order : {e}")
                


               
            return create_take_profit_order

        except Exception as e:
            # Handle any exceptions that may occur during execution
            print(f"Error in send_2nd_post_only_order: {str(e)}")
            return None, None  # Return None for both orders in case of an error



    def close_open_orders(self):
        open_orders = self.exchange.fetch_open_orders(self.symbol)
        for order in open_orders:
            self.exchange.cancel_order(order['id'], self.symbol)
        return open_orders

    def close_position_with_market_order(self):
        position = self.exchange.fetch_positions([self.symbol])
        quantity = position[0]['info']['positionAmt']
        side = 'sell' if position[0]['side'] == 'long' else 'buy'
        quantity = abs(float(quantity))
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': True
            }
        )
        return order
    def close_position_and_orders(self):
        self.close_open_orders()
        self.close_position_with_market_order()
test = SendOrders(exchange, symbol)
test.calculate_quantity()


6.002184795265477

In [106]:
# test.close_open_orders()
# test.close_position_and_orders()
test.create_1st_market_order('buy')
test.create_2nd_tp_and_sl_order()


quantity: 6.055


{'info': {'orderId': '1180951213',
  'symbol': 'ETHUSDT',
  'status': 'NEW',
  'clientOrderId': 'x-xcKtGhcu40a6815f7192143257686b',
  'price': '1652.48',
  'avgPrice': '0.00',
  'origQty': '6.055',
  'executedQty': '0.000',
  'cumQty': '0.000',
  'cumQuote': '0.00000',
  'timeInForce': 'GTX',
  'type': 'LIMIT',
  'reduceOnly': True,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '0.00',
  'workingType': 'CONTRACT_PRICE',
  'priceProtect': False,
  'origType': 'LIMIT',
  'priceMatch': 'NONE',
  'selfTradePreventionMode': 'NONE',
  'goodTillDate': '0',
  'updateTime': '1696346991616'},
 'id': '1180951213',
 'clientOrderId': 'x-xcKtGhcu40a6815f7192143257686b',
 'timestamp': 1696346991616,
 'datetime': '2023-10-03T15:29:51.616Z',
 'lastTradeTimestamp': None,
 'lastUpdateTimestamp': 1696346991616,
 'symbol': 'ETH/USDT:USDT',
 'type': 'limit',
 'timeInForce': 'PO',
 'postOnly': True,
 'reduceOnly': True,
 'side': 'sell',
 'price': 1652.48,
 'triggerPrice

In [104]:
test.close_position_and_orders()
